# Proof of Concepts

## Libraries, Global Variables, Utility Functions

In [43]:
import subprocess
import re
import numpy as np
import random
from Bio.PDB import *

random.seed(10)

repository_path = "/Users/albertdarmawan/Documents/gass-ppi/"
dbd5_path = repository_path + "dataset/benchmark5/structures/"
pdb_parser = PDBParser()

# List of PDB ID of protein complexes
# aa stands for Antibody-Antigen (in this case, Antibody is the receptor and Antigen is the ligand)
dbd5_aa_list = ["1AHW", "1BVK", "1DQJ", "1E6J", "1JPS", "1MLC", "1VFB", "1WEJ",
                              "2FD6", "2I25", "2VIS", "2VXT", "2W9E", "3EOA", "3HMX", "3MXW",
                              "3RVW", "4DN4", "4FQI", "4G6J", "4G6M", "4GXU", "3EO1", "3G6D",
                              "3HI6", "3L5W", "3V6Z", "1BGX"]

dbd5_aa_r_list = list(map(lambda x: x + "_r_u", dbd5_aa_list))
dbd5_aa_l_list = list(map(lambda x: x + "_l_u", dbd5_aa_list))
# example = pdb_parser.get_structure("1AHW_L", dataset_directory + "1AHW_l_u.pdb")
lha_dict = {
    "GLY":"CA",
    "ALA":"CB",
    "GLN":"CD",
    "GLU":"CD",
    "ILE":"CD1",
    "LEU":"CD1",
    "MET":"CE",
    "HIS":"CE1",
    "ASN":"CG",
    "ASP":"CG",
    "PRO":"CG",
    "VAL":"CG1",
    "THR":"CG2",
    "TRP":"CH2",
    "ARG":"CZ",
    "PHE":"CZ",
    "LYS":"NZ",
    "SER":"OG",
    "TYR":"OH",
    "CYS":"SG",
}

In [44]:
# In this case, a Residue is represented by a reference atom (LHA, CA, etc)
# Residue is the "gene" in genetic algorithms.
# Therefore, a list of Residue corresponds to an "individual"
# A residue consists of 2 residue information, 1 chain information, and 2 reference atom information
class Residue:
    def __init__(self, residue_name, residue_sequence_position, chain_name, atom_name, atom_coordinates):
        self.residue_name = residue_name
        self.residue_sequence_position = residue_sequence_position
        self.chain_name = chain_name
        self.atom_name = atom_name
        self.atom_coordinates = atom_coordinates

In [45]:
def print_interface_info(interface):
    """Print Interface Info
    Given an interface, print relevant information
    For debugging purpose only
    
    Parameters:
    interface (list): The list of Residue object which constitutes the interface
    
    Returns:
    None
    
    """
    print("Number of Residues: ", len(interface))
    for i in range(len(interface)):
        if i > 0:
            print(" - ", end="")
        print(interface[i].residue_name, interface[i].residue_sequence_position, interface[i].chain_name, end="")
    print("")

In [46]:
def euclidean_distance(coordinate_1, coordinate_2):
    """Euclidean Distance
    Given 3-dimensional coordinates of 2 atoms, calculate its Euclidean distance
    
    Parameters:
    coordinate_1 (1D NumPy Array): x,y,z coordinates of the first atom
    coordinate_2 (1D NumPy Array): x,y,z coordinates of the second atom
    
    Returns:
    float: The euclidean distance
    
    """
    return float(np.sqrt(((coordinate_1[0] - coordinate_2[0]) ** 2) +
                   ((coordinate_1[1] - coordinate_2[1]) ** 2) +
                   ((coordinate_1[2] - coordinate_2[2]) ** 2)))

In [47]:
def load_pdb(pdb_id, pdb_directory_path, pdb_parser, lha_dict, reference_atom="lha"):
    """Load PDB
    Given a PDB ID and its directory, load the .pdb file using Bio.PDB module and generate a list of residue
    
    Parameters:
    pdb_id (str): The PDB ID for the protein structure
    pdb_directory_path (str): Absolute path to access the PDB file
    pdb_parser (Bio.PDB.PDBParser.PDBParser): Bio.PDB Parser
    lha_dict (dict): Corresponding Last Heavy Atom for each amino acids 
    reference_atom (str): Reference atom used ("lha" or "ac")
    
    Returns:
    list: List of Residue object which constitutes the protein structure
    
    """
    residue_list = []
    amino_acid_list = list(lha_dict.keys())
    pdb_file_path = pdb_directory_path + pdb_id + ".pdb"
    pdb_structure = pdb_parser.get_structure(pdb_id, pdb_file_path)
    # Only take the ATOM keyword (exclude the HETATM, hetero atom that is not inside standard amino acids)
    biopdb_residue_list = [residue for residue in pdb_structure.get_residues() if residue.get_resname() in amino_acid_list]
    if reference_atom == "lha":
        biopdb_atom_list = [atom for residue in biopdb_residue_list for atom in residue if atom.get_name() == lha_dict[residue.get_resname()]]
    else:
        biopdb_atom_list = [atom for residue in biopdb_residue_list for atom in residue if atom.get_name() == "CA"]
        
    for atom in biopdb_atom_list:
        # Create a new Residue instance
        current_residue_name = atom.get_parent().get_resname()
        current_residue_sequence_position = atom.get_parent().get_full_id()[3][1]
        current_chain_name = atom.get_parent().get_parent().id
        current_atom_name = atom.get_name()
        current_atom_coordinates = atom.get_coord()
        current_atom = Residue(current_residue_name,
                               current_residue_sequence_position,
                               current_chain_name,
                               current_atom_name,
                               current_atom_coordinates)
        residue_list.append(current_atom)
    return residue_list

residue_3nos = load_pdb("1AHW_r_u", dbd5_path, pdb_parser, lha_dict, "lha")
print(type(residue_3nos))
print(len(residue_3nos))
for item in residue_3nos[:5]:
    print(item.residue_name)
    print(item.residue_sequence_position)
    print(item.chain_name)
    print(item.atom_name)
    print(item.atom_coordinates)

<class 'list'>
428
ASP
1
L
CG
[-9.708  7.201 49.865]
ILE
2
L
CD1
[-2.641  4.83  49.435]
LYS
3
L
NZ
[-2.375 14.861 47.739]
MET
4
L
CE
[ 1.65   3.66  45.774]
THR
5
L
CG2
[ 6.169 10.366 47.363]


/opt/anaconda3/envs/thesis/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 3305
  warnings.warn(


In [48]:
def tmalign_structural_alignment(pdb_id_1, pdb_id_2, pdb_directory_path):
    """TMAlign Structural Alignment
    Compare structural similarities between two PDB structures (regardless of the rotation)
    https://zhanggroup.org/TM-score/
    
    Parameters:
    pdb_id_1 (str): The PDB ID for the first protein structure
    pdb_id_2 (str): The PDB ID for the second protein structure
    pdb_directory_path (str): Absolute path to access the PDB files
    
    Returns:
    float: The TMScore, a value between (0,1]. 1 indicates a perfect match. >0.5 is similar enough. <0.17 is two unrelated structures.
    
    """
    pdb_id_1_file_path = pdb_directory_path + pdb_id_1 + ".pdb"
    pdb_id_2_file_path = pdb_directory_path + pdb_id_2 + ".pdb"
    # Execute TMAlign
    tmalign_thread = subprocess.run(["./TMalign", pdb_id_1_file_path, pdb_id_2_file_path], capture_output=True, text=True)
    output_text = tmalign_thread.stdout

    # Retrieved TMScore from TMAlign results
    tmscore_raw_list = re.findall("TM-score=\s[0-9]+.[0-9]+", output_text)

    # Convert the TMScore into floats, then get the maximum TMScore
    tmscore_list = list(map(lambda x: float(re.sub("TM-score=\s", "", x)), tmscore_raw_list))
    max_tmscore = max(tmscore_list)
    return max_tmscore

tmscore = tmalign_structural_alignment("1AHW_l_u", "1BVK_l_u", dbd5_path)
print(tmscore)

0.23652


In [49]:
def get_actual_interface(residue_list_1, residue_list_2, threshold=6.0):
    """Get Actual Interface
    Given one receptor structure and one ligand structure, infer its interfaces based on certain threshold
    
    Parameters:
    residue_list_1 (list): List of Residue object from the first protein structure
    residue_list_2 (list): List of Residue object from the second protein structure
    threshold (float): The acceptable distance between a receptor's atom and a ligand's atom
    
    Returns:
    list: List of Residue object which constitutes the protein interface
    
    """
    interface_list = []
    for residue_1 in residue_list_1:
        for residue_2 in residue_list_2:
            current_distance = euclidean_distance(residue_1.atom_coordinates, residue_2.atom_coordinates)
            if current_distance < threshold:
                interface_list.append(residue_1)
                break

    return interface_list

ligand = load_pdb("1AHW_l_u", dbd5_path, pdb_parser, lha_dict, "lha")
receptor = load_pdb("1AHW_r_u", dbd5_path, pdb_parser, lha_dict, "lha")
sample_interface_list = get_actual_interface(ligand, receptor)
print(sample_interface_list)
print(type(sample_interface_list))
print(len(sample_interface_list))
for item in sample_interface_list:
    print(item.residue_name)

/opt/anaconda3/envs/thesis/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 1623
  warnings.warn(
/opt/anaconda3/envs/thesis/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 3305
  warnings.warn(


[<__main__.Residue object at 0x7fa896e17a60>, <__main__.Residue object at 0x7fa896ddb4c0>, <__main__.Residue object at 0x7fa896ddb520>, <__main__.Residue object at 0x7fa896ddb5e0>, <__main__.Residue object at 0x7fa896ddb6a0>, <__main__.Residue object at 0x7fa896ddb760>, <__main__.Residue object at 0x7fa896ddb7c0>, <__main__.Residue object at 0x7fa896ddbb20>, <__main__.Residue object at 0x7fa896ddbb80>, <__main__.Residue object at 0x7fa896ddbbe0>, <__main__.Residue object at 0x7fa896ddbc40>, <__main__.Residue object at 0x7fa896ddbca0>, <__main__.Residue object at 0x7fa896ddbd00>, <__main__.Residue object at 0x7fa896ddbd60>, <__main__.Residue object at 0x7fa896ddbfa0>, <__main__.Residue object at 0x7fa896ebf460>, <__main__.Residue object at 0x7fa896ebf520>, <__main__.Residue object at 0x7fa896ebf5e0>, <__main__.Residue object at 0x7fa896ebf640>, <__main__.Residue object at 0x7fa896ebf700>, <__main__.Residue object at 0x7fa896ebf820>, <__main__.Residue object at 0x7fa896ebf880>, <__main__

## GASS-PPI Core Functions

In [56]:
def calculate_fitness_score(individual, interface_template):
    """Calculate Fitness Score
    Adhering to the original GASS, calculate the spatial LHA distance between an individual and the interface template
    Essentially, it's the modified version of RMSD
    
    Parameters:
    individual (list): The individual that needs to be evaluated
    interface_template (list): The interface template, used as a reference
    
    Returns:
    float: The fitness score of an individual
    
    """
    return 0.1

In [51]:
def crossover(individual_1, individual_2):
    """Crossover
    Given two individuals, perform a single point crossover
    
    Parameters:
    individual_1 (list): First individual
    individual_2 (list): Second individual
    
    Returns:
    list: Individual 1 after crossover
    list: Individual 2 after crossover
    
    """
    individual_size = len(individual_1)
    crossover_point = random.randrange(individual_size)
    
    # Swap residues between two individuals, starting from the crossover point
    for i in range(crossover_point, individual_size):
        individual_1[i], individual_2[i] = individual_2[i], individual_1[i]
        
    return (individual_1, individual_2)

In [52]:
def mutation(input_protein_structure, individual):
    """Mutation
    Given an individual, perform a single point mutations
    In this case, no conservative mutation is employed
    
    Parameters:
    input_protein_structure (list): List of Residues object which constitutes the input protein structure
    individual (list): Individual to be mutated
    
    Returns:
    list: Mutated individuals
    
    """
    individual_size = len(individual)
    mutation_point = random.randrange(individual_size)
    mutated_residue = individual[mutation_point]
    
    # Mutate the residue at the mutation point into the same amino acid types 
    possible_mutation = [residue for residue in input_protein_structure 
                         if residue.residue_name == mutated_residue.residue_name 
                         and residue.residue_sequence_position != mutated_residue.residue_sequence_position]
    
    new_residue_index = random.randrange(len(possible_mutation))
    new_residue = possible_mutation[new_residue_index]
    
    individual[mutation_point] = new_residue
    return individual


In [57]:
def gass_ppi(input_protein_structure, interface_template):
    """GASS-PPI Method
    Given the input protein structure and the interface template, perform genetic algorithms
    to search the most plausible interface
    
    Parameters:
    input_protein_structure (list): 
    interface_template (list): 
    
    Returns:
    list: 
    
    """
    template_size = len(interface_template)
    # List of hyperparameters (look at GASS' Supplementary Material, Table 2)
    population_size = 150
    number_of_generations = 100
    crossover_probability = 0.9
    mutation_probability = 0.2
    
    # Generate Initial Population
    input_protein_structure_copy = list(input_protein_structure)
    initial_population = []
    for _ in range(population_size):
        random.shuffle(input_protein_structure_copy)
        initial_population.append(input_protein_structure_copy[:template_size])
    
    print("")
    print("Initial Population")
    print(len(initial_population))
    print_interface_info(initial_population[0])
    print_interface_info(initial_population[1])
    print("")
    
    # Initially, perform fitness evaluation
    fitness_score_list = [calculate_fitness_score(individual, interface_template) for individual in initial_population]
    print(len(fitness_score_list))
    print(fitness_score_list)
    
    # Evolutionary Steps
    for i in range(number_of_generations):
        # Fitness evaluation
        break
        # Selection (deterministic tournament)
        
        # Crossover
        
        # Mutation
        
    # Return the most probable individuals
    
    return 0

In [54]:
def evaluate(predicted_interface, actual_interface):
    """Evaluate
    Given the predicted interface and its actual interface, calculate some performance metrics
    
    Parameters:
    predicted_interface (list): 
    actual_inteface (list): 
    
    Returns:
    float: Precision score
    float: Recall score 
    float: AUC-ROC score
    float: AUC-PR score
    
    """
    # True Positive (TP)
    tp = 5
    # True Negative (TN)
    tn = 6
    # False Positive (FP)
    fp = 7
    # False Negative (FN)
    fn = 8
    
    # Evaluation Metrics
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    auc_roc_score = 0.7
    auc_pr_score = 0.8
    return (precision, recall, auc_roc_score, auc_pr_score)

## Main Program

In [58]:
# Query Protein
input_pdb_id = "1AHW_r_u"
input_pdb_structure = load_pdb(input_pdb_id, dbd5_path, pdb_parser, lha_dict, "lha")
print("Input PDB ID: ", input_pdb_id)
print("Number of Residues: ", len(input_pdb_structure))
print("")
# Remove the first element in this case, since it's the input_pdb_id
dataset_list = dbd5_aa_r_list[1:]

# Step 1: Find the structural neighbour of the query protein using TMAlign
tmscore_list = list(map(lambda x: tmalign_structural_alignment(input_pdb_id, x, dbd5_path), dataset_list))
maximum_index = np.argmax(tmscore_list)
neighbour_pdb_id = dataset_list[maximum_index]
print("Neighbour PDB ID: ", neighbour_pdb_id)
print("Alignment Score: ", tmscore_list[maximum_index])

# Step 2: Find the interface of the structural neighbour, then use it as the interface template
neighbour_pair_pdb_id = neighbour_pdb_id[:5] + ("l" if neighbour_pdb_id[5] == "r" else "r") + neighbour_pdb_id[6:]

neighbour_pdb_structure = load_pdb(neighbour_pdb_id, dbd5_path, pdb_parser, lha_dict, "lha")
neighbour_pair_pdb_structure = load_pdb(neighbour_pair_pdb_id, dbd5_path, pdb_parser, lha_dict, "lha")
print("Number of Residues: ", len(neighbour_pdb_structure))
print("")
print("Neighbour Pair PDB ID: ", neighbour_pair_pdb_id)
print("Number of Residues: ", len(neighbour_pair_pdb_structure))
print("")

interface_template = get_actual_interface(neighbour_pdb_structure, neighbour_pair_pdb_structure)
print("Interface Template")
print_interface_info(interface_template)

# Step 3: GASS-PPI
predicted_interface = gass_ppi(input_pdb_structure, interface_template)

# Step 4: Evaluation
input_pair_pdb_id = input_pdb_id[:5] + ("l" if input_pdb_id[5] == "r" else "r") + input_pdb_id[6:]
input_pair_pdb_structure = load_pdb(input_pair_pdb_id, dbd5_path, pdb_parser, lha_dict, "lha")
actual_interface = get_actual_interface(input_pdb_structure, input_pair_pdb_structure)
print("Input Pair PDB ID: ", input_pair_pdb_id)
print("Number of Residues: ", len(input_pair_pdb_structure))
print("")
print("Actual Interface")
print_interface_info(actual_interface)

precision, recall, auc_roc_score, auc_pr_score = evaluate(predicted_interface, actual_interface)
print("Evaluation")
print("Precision: ", precision)
print("Recall: ", recall)
print("AUC-ROC Score: ", auc_roc_score)
print("AUC-PR Score: ", auc_pr_score)

/opt/anaconda3/envs/thesis/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 3305
  warnings.warn(


Input PDB ID:  1AHW_r_u
Number of Residues:  428

Neighbour PDB ID:  3MXW_r_u
Alignment Score:  0.97342
Number of Residues:  431

Neighbour Pair PDB ID:  3MXW_l_u
Number of Residues:  161

Interface Template
Number of Residues:  23
SER 30 L - ASN 31 L - ASP 32 L - TYR 49 L - TYR 50 L - ASN 53 L - ARG 54 L - TYR 55 L - THR 56 L - ASP 91 L - GLY 93 L - ASP 31 H - GLU 32 H - ALA 33 H - ARG 52 H - TYR 54 H - ARG 94 H - ASP 95 H - TRP 96 H - GLU 97 H - ARG 98 H - GLY 99 H - TYR 102 H

Initial Population
150
Number of Residues:  23
TYR 94 H - ASN 210 L - LYS 207 L - GLU 154 L - SER 75 H - ASN 57 H - GLN 82 H - SER 208 L - ASP 110 L - SER 76 L - ALA 112 L - ILE 70 H - ALA 25 L - LYS 142 L - LEU 83 H - ILE 150 L - SER 7 H - THR 202 L - ILE 144 L - THR 172 L - PRO 122 H - ARG 61 L - SER 43 L
Number of Residues:  23
LYS 209 H - LEU 11 H - LYS 119 H - ARG 155 L - ALA 9 H - ASP 143 L - GLN 109 H - ASP 151 L - MET 4 L - PRO 40 L - TYR 140 L - PHE 150 H - LEU 83 H - TYR 96 L - THR 164 L - SER 189 H 

/opt/anaconda3/envs/thesis/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 1623
  warnings.warn(


Input Pair PDB ID:  1AHW_l_u
Number of Residues:  202

Actual Interface
Number of Residues:  21
GLN 27 L - TYR 32 L - TYR 50 L - GLU 93 L - SER 94 L - PRO 95 L - TYR 96 L - ASP 31 H - TYR 32 H - TYR 33 H - LEU 50 H - ASP 52 H - ASN 55 H - ASN 57 H - ILE 59 H - PRO 62 H - ARG 98 H - ASP 99 H - ASN 100 H - SER 101 H - TYR 102 H
Evaluation
Precision:  0.4166666666666667
Recall:  0.38461538461538464
AUC-ROC Score:  0.7
AUC-PR Score:  0.8


In [38]:
sample_individual_1 = actual_interface[:10]
sample_individual_2 = actual_interface[10:20]
print_interface_info(sample_individual_1)
# print_interface_info(sample_individual_2)

# child_1, child_2 = crossover(sample_individual_1, sample_individual_2)
# print_interface_info(child_1)
# print_interface_info(child_2)

mutated_individual = mutation(input_pdb_structure, sample_individual_1)
print_interface_info(mutated_individual)

Number of Residues:  10
GLN 27 L - TYR 32 L - TYR 50 L - GLU 93 L - SER 94 L - PRO 95 L - TYR 96 L - ASP 31 H - TYR 32 H - TYR 33 H
Number of Residues:  10
GLN 27 L - TYR 32 L - TYR 50 L - GLU 93 L - SER 94 L - PRO 95 L - TYR 96 L - ASP 73 H - TYR 32 H - TYR 33 H
